# **NLP4_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Mathematical Optimization Model

Parameters
- Revenue per barrel of alkylate yield = r
- Cost per barrel of olefin feed = oc
- Cost per barrel of isobutane recycle = irc
- Cost per thousand pounds of acid addition = aac
- Cost per barrel of isobutane makeup = imc

Decision Variables
- Olefin feed (OF) in barrels/day
- Isobutane recycle (IR) in barrels/day
- Acid addition rate (AR) in thousands pounds/day
- Isobutane Makeup (IM) in barrels/day

Derived Variables
- Alkylate yield (AY) in barrels/day
- Motor Octane number (MO)
- Acid Dilution Factor (ADF)
- F-4 Performance Number (FP)

Objective Function
Maximize Profit = AY*r*MO - (oc*OF + irc*IR + aac*AR + imc*IM)

Constraints
1. AY = OF * (1.12 + 0.13167 * (IR+IM/OF) - 0.00667 * (IR+IM/OF)^2) ± 2%
2. MO = 86.35 + 1.098 * (IR+IM/OF) - 0.038 * (IR+IM/OF)^2 + 0.325 * ((93000*AR)/(AR*ADF + 1000*AR)) - 89 ± 1%
3. ADF = 35.82 - 0.222*FP ± 5%
4. FP = -133 + 3*MO ± 10%
5. Theme of acceptable range of values for decision variables (OF, IR, AR, 

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is a Pyomo code for the problem you specified, using some sample data. 

```python
from pyomo.environ import *

#Create Concrete Model 
model = ConcreteModel()

#Set the Parameters
model.r = Param(initialize=5.0) #Revenue per barrel of alkylate yield
model.oc = Param(initialize=2.0) #Cost per barrel of olefin feed
model.irc = Param(initialize=3.0) #Cost per barrel of isobutane recycle
model.aac = Param(initialize=4.0) #Cost per thousand pounds of acid addition
model.imc = Param(initialize=1.5) #Cost per barrel of isobutane makeup

#Set the Decision variables
model.OF = Var(initialize=5.0,within=NonNegativeReals)
model.IR = Var(initialize=30,within=NonNegativeReals)
model.AR = Var(initialize=22,within=NonNegativeReals)
model.IM = Var(initialize=40,within=NonNegativeReals)

#Set the Derived Variables
model.AY = Var(initialize=105,within=NonNegativeReals)
model.MO = Var(initialize=90)
model.ADF = Var(initialize=35)
model.FP = Var(initialize=-100)

#Define the Objective
def profit(model):
    return (model.AY*model.r*model.MO - (model.oc*model.OF + model.irc*model.IR + model.aac*model.AR + model.imc*model.IM))
model.profit = Objective(rule=profit, sense=maximize)

#Define the Constraints
def cstr_AY(model):
    return model.AY == model.OF * (1.12 + 0.13167 * (model.IR+model.IM/model.OF) - 0.00667 * (model.IR+model.IM/model.OF)**2)
model.constraint1 = Constraint(rule=cstr_AY) 

def cstr_MO(model):
    return model.MO == 86.35 + 1.098 * (model.IR+model.IM/model.OF) - 0.038 * (model.IR+model.IM/model.OF)**2 + 0.325 * ((93000*model.AR)/(model.AR*model.ADF + 1000*model.AR)) - 89
model.constraint2 = Constraint(rule=cstr_MO) 

def cstr_ADF(model):
    return model.ADF == 35.82 - 0.222*model.FP
model.constraint3 = Constraint(rule=cstr_ADF)

def cstr_FP(model):
    return model.FP == -133 + 3*model.MO
model.constraint4 = Constraint(rule=cstr_FP)

def eqn_IM_OF(model):
    return 1.22*model.AY == model.OF + model.IM
model.constraint5 = Constraint(rule=eqn_IM_OF)

#Solver 
SolverFactory('ipopt').solve(model)

#printing solution
model.OF, model.IR, model.AR, model.IM, model.AY, model.MO, model.ADF, model.FP, model.profit()

```
Note: The problem that you have posed might not be a feasible one. Please make sure that the constraints form a feasible region. You may need to adjust the constraints, or the parameters, in order to get a non-empty feasible region. Plus, the equation `(IR+IM/OF) = external isobutane-to-olefin ratio` might not be used as the value for the external isobutane-to-olefin ratio was not provided. Please include it when you try to run the model for your actual problem.

### **Run the code Generated by GPT4**

In [9]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.6 MB/s eta 0:00:00


In [10]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [14]:
from pyomo.environ import *

# Create Concrete Model
model = ConcreteModel()

# Set the Parameters
model.r = Param(initialize=5.0) #Revenue per barrel of alkylate yield
model.oc = Param(initialize=2.0) #Cost per barrel of olefin feed
model.irc = Param(initialize=3.0) #Cost per barrel of isobutane recycle
model.aac = Param(initialize=4.0) #Cost per thousand pounds of acid addition
model.imc = Param(initialize=1.5) #Cost per barrel of isobutane makeup

# Set the Decision variables
model.OF = Var(initialize=5.0,within=NonNegativeReals)
model.IR = Var(initialize=30,within=NonNegativeReals)
model.AR = Var(initialize=22,within=NonNegativeReals)
model.IM = Var(initialize=40,within=NonNegativeReals)

# Set the Derived Variables
model.AY = Var(initialize=105,within=NonNegativeReals)
model.MO = Var(initialize=90)
model.ADF = Var(initialize=35)
model.FP = Var(initialize=-100)

# Define the Objective
def profit(model):
    return (model.AY*model.r*model.MO - (model.oc*model.OF + model.irc*model.IR + model.aac*model.AR + model.imc*model.IM))
model.profit = Objective(rule=profit, sense=maximize)

# Define the Constraints
def cstr_AY(model):
    return model.AY == model.OF * (1.12 + 0.13167 * (model.IR+model.IM/model.OF) - 0.00667 * (model.IR+model.IM/model.OF)**2)
model.constraint1 = Constraint(rule=cstr_AY)

def cstr_MO(model):
    return model.MO == 86.35 + 1.098 * (model.IR+model.IM/model.OF) - 0.038 * (model.IR+model.IM/model.OF)**2 + 0.325 * ((93000*model.AR)/(model.AR*model.ADF + 1000*model.AR)) - 89
model.constraint2 = Constraint(rule=cstr_MO)

def cstr_ADF(model):
    return model.ADF == 35.82 - 0.222*model.FP
model.constraint3 = Constraint(rule=cstr_ADF)

def cstr_FP(model):
    return model.FP == -133 + 3*model.MO
model.constraint4 = Constraint(rule=cstr_FP)

def eqn_IM_OF(model):
    return 1.22*model.AY == model.OF + model.IM
model.constraint5 = Constraint(rule=eqn_IM_OF)

# Solver
SolverFactory('knitro').solve(model)

# printing solution
print(model.OF(), model.IR(), model.AR(), model.IM(), model.AY(), model.MO(), model.ADF(), model.FP(), model.profit())

  - termination condition: optimal
  - message from solver: Knitro 14.0.0\x3a Relative change in feasible objective < ftol for ftol_iters.; objective 6413365136617472000; feasibility error 0; 329 iterations; 871 function evaluations


2.193370229968316e+16 8.718306106558282 4.7230875156620976e-07 2.542489933194117e+16 3.881852592756093e+16 33.46530933367089 43.05810398377519 -32.60407199898735 6.413365136617472e+18


In [16]:
from pyomo.environ import *

# Create Concrete Model
model = ConcreteModel()

# Set the Parameters - data inputted by human
model.r = Param(initialize=0.63) #Revenue per barrel of alkylate yield
model.oc = Param(initialize=5.04) #Cost per barrel of olefin feed
model.irc = Param(initialize=0.035) #Cost per barrel of isobutane recycle
model.aac = Param(initialize=10.0) #Cost per thousand pounds of acid addition
model.imc = Param(initialize=3.36) #Cost per barrel of isobutane makeup

# Set the Decision variables
model.OF = Var(initialize=5.0,within=NonNegativeReals)
model.IR = Var(initialize=30,within=NonNegativeReals)
model.AR = Var(initialize=22,within=NonNegativeReals)
model.IM = Var(initialize=40,within=NonNegativeReals)

# Set the Derived Variables
model.AY = Var(initialize=105,within=NonNegativeReals)
model.MO = Var(initialize=90)
model.ADF = Var(initialize=35)
model.FP = Var(initialize=-100)

# Define the Objective
def profit(model):
    return (model.AY*model.r*model.MO - (model.oc*model.OF + model.irc*model.IR + model.aac*model.AR + model.imc*model.IM))
model.profit = Objective(rule=profit, sense=maximize)

# Define the Constraints
def cstr_AY(model):
    return model.AY == model.OF * (1.12 + 0.13167 * (model.IR+model.IM/model.OF) - 0.00667 * (model.IR+model.IM/model.OF)**2)
model.constraint1 = Constraint(rule=cstr_AY)

def cstr_MO(model):
    return model.MO == 86.35 + 1.098 * (model.IR+model.IM/model.OF) - 0.038 * (model.IR+model.IM/model.OF)**2 + 0.325 * ((93000*model.AR)/(model.AR*model.ADF + 1000*model.AR)) - 89
model.constraint2 = Constraint(rule=cstr_MO)

def cstr_ADF(model):
    return model.ADF == 35.82 - 0.222*model.FP
model.constraint3 = Constraint(rule=cstr_ADF)

def cstr_FP(model):
    return model.FP == -133 + 3*model.MO
model.constraint4 = Constraint(rule=cstr_FP)

def eqn_IM_OF(model):
    return 1.22*model.AY == model.OF + model.IM
model.constraint5 = Constraint(rule=eqn_IM_OF)

# Solver
SolverFactory('knitro').solve(model)

# printing solution
print(model.OF(), model.IR(), model.AR(), model.IM(), model.AY(), model.MO(), model.ADF(), model.FP(), model.profit())

7.999638385787553e-11 2.374132860712966e-07 1.0004699827774964e-09 9.600420261848818e-10 6.41547422662128e-10 34.04216842709386 42.67391582755544 -30.87349471871822 -8.184098610668994e-09


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**